In [ ]:
for i in range(5, 7):
    print(i)

In [8]:
def sort_month(months):
    if 3 in months and 12 in months:
        _months = [m+12 for m in months if m <=6] + [m for m in months if m > 6]
        _months = sorted(_months)
        return [m for m in _months if m <= 12]+[m-12 for m in _months if m > 12]
    else:
        return sorted(months)

In [10]:
a = [[1,2,3,6], [2,3,6,9], [3,4,6,9],[4,5,6,9],[5,6,9,12],[6,7,9,12],[7,8,9,12],[8,9,12,3],[9,10,12,3],[10,11,12,3],[11,12,3,6], [12, 1, 3, 6]]
for i in a:
    print(sort_month(i))

[1, 2, 3, 6]
[2, 3, 6, 9]
[3, 4, 6, 9]
[4, 5, 6, 9]
[5, 6, 9, 12]
[6, 7, 9, 12]
[7, 8, 9, 12]
[8, 9, 12, 3]
[9, 10, 12, 3]
[10, 11, 12, 3]
[11, 12, 3, 6]
[12, 1, 3, 6]
